In [1]:
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
from tqdm import tqdm

In [2]:
import pandas as pd

In [4]:
torch.cuda.is_available()

True

In [30]:
df = pd.read_csv('/h/brandon/internship/Uncover_implicit_bias/gpt2_training/data/ROCStories_winter2017.csv')

In [31]:
df = df.head(20)

In [32]:
# Define the function to concatenate the story title with the sentences
def concatenate_story_elements(row):
    title = row['storytitle']
    sentences = ' '.join(row[['sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5']].tolist())
    return f'{title} <SEP> {sentences}'

# Apply the function to each row
df['story'] = df.apply(concatenate_story_elements, axis=1)

stories_data = list(df['story'])




In [37]:
# Convert list of strings to a dictionary with a key
stories_dict = {'story': stories_data}

# Convert the dictionary to a Hugging Face Dataset
dataset = Dataset.from_dict(stories_dict)


Format = story_title <SEP> story_text

p(story_text | story_title <SEP>)


Format2 = <title> story_title <s1> s1 <s2> s2 ... 

p(rest of sentences|<title> story_title)

In [5]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [6]:
tokenizer.add_tokens(['<TITLE>'])
tokenizer.add_tokens(['<SEP>'])

1

In [26]:
token_id = tokenizer.convert_tokens_to_ids('<SEP>')  # Replace <TITLE> with your token
print(f"The id for <SEP> is: {token_id}")

The id for <SEP> is: 50256


In [25]:
# Convert id to token
token_id = 50257
token = tokenizer.convert_ids_to_tokens(token_id)
print(f"The token for id {token_id} is: {token}")

The token for id 50257 is: <TITLE>


## Load ROCStories

In [9]:
# Tokenize the input data
train_path = '/h/brandon/internship/Uncover_implicit_bias/gpt2_training/data/ROCStories_winter2017.csv'  # This file should contain one story per line
val_path = 'path_to_validation_data.txt'  # This file should contain one story per line

# Create a dataset and collator
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128
)

/h/brandon/.local/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [42]:
tokenizer(dataset[0]['story'])

{'input_ids': [11006, 41692, 262, 14331, 50258, 11006, 6810, 339, 550, 1234, 319, 257, 1256, 286, 3463, 2904, 13, 679, 11068, 465, 13870, 284, 1949, 290, 3785, 503, 262, 1738, 13, 679, 6939, 339, 1549, 587, 6600, 1165, 881, 3049, 2057, 16537, 13, 679, 5025, 1016, 284, 26593, 4113, 290, 2067, 257, 24053, 5496, 13, 2293, 257, 1178, 2745, 11, 339, 2067, 284, 1254, 881, 1365, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# Define a function to apply to each example
def preprocess_function(examples):
    # Modify examples here (e.g., tokenize text)
    return examples

# Apply the function to the dataset
processed_dataset = dataset.map(preprocess_function)

In [41]:
# Function to decode the token IDs
def decode_sample(token_ids):
    return tokenizer.decode(token_ids, clean_up_tokenization_spaces=True, skip_special_tokens=True)

# Visualize the first few samples from train_dataset
print("Some samples from the training set:\n")
for i in range(5):  # let's print out the first 5 samples
    token_ids = dataset[i]['story']  # get the token IDs for the i-th sample
    text = decode_sample(token_ids)  # decode the token IDs to text
    print(f"Sample {i + 1}:\n{text}\n")

Some samples from the training set:



ValueError: invalid literal for int() with base 10: 'D'

## Train

In [ ]:


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',         # Output directory
    overwrite_output_dir=True,      # Overwrite the content of the output directory
    num_train_epochs=3,             # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    per_device_eval_batch_size=4,   # Batch size for evaluation
    eval_steps=400,                 # Evaluation step
    save_steps=800,                 # After # steps model is saved
    warmup_steps=500,               # Warmup steps
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("fine_tuned_gpt2_rocstories")